In [282]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV, ShuffleSplit, train_test_split

properati = pd.read_csv('datos/caba_para_mapa.csv',error_bad_lines=False)

<IPython.core.display.Javascript object>

# RF Clasificador Casas

In [283]:
properati = properati.loc[properati['property_type'] == 'house']
properati.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2751 entries, 4 to 48322
Data columns (total 26 columns):
created_on                    2751 non-null object
property_type                 2751 non-null object
place_name                    2751 non-null object
state_name                    2751 non-null object
lat-lon                       2751 non-null object
lat                           2751 non-null float64
lon                           2751 non-null float64
price                         2751 non-null float64
currency                      2751 non-null object
price_aprox_local_currency    2751 non-null float64
price_aprox_usd               2751 non-null float64
surface_total_in_m2           2751 non-null float64
surface_covered_in_m2         2751 non-null float64
price_usd_per_m2              2751 non-null float64
price_per_m2                  2751 non-null float64
floor                         1 non-null float64
rooms                         1134 non-null float64
expenses         

In [284]:
#paso features con strings a numericos
le_barrio = preprocessing.LabelEncoder()
barrios=properati['place_name']
le_barrio.fit(barrios)
properati['place_name'] = le_barrio.transform(barrios)

le_divisa = preprocessing.LabelEncoder()
divisa=properati['currency']
le_divisa.fit(divisa)
properati['currency'] = le_divisa.transform(divisa)

le_tipo = preprocessing.LabelEncoder()
tipos_prop=properati['property_type']
le_tipo.fit(tipos_prop)
properati['property_type'] = le_tipo.transform(tipos_prop)

###  Agrego features

### Pruebo con np.histogram

In [289]:
cant_buckets = 100

#buckets
cantidad,rango = np.histogram(properati['price_per_m2'], bins=cant_buckets)
print rango
properati['categories_by_price']=pd.cut(properati['price_per_m2'],rango,labels=np.arange(cant_buckets),include_lowest=True)
properati['price_range']=pd.cut(properati['price_per_m2'],rango)

properati.loc[properati['categories_by_price']!=properati['categories_by_price']]
#properati['categories_by_price']=pd.qcut(x=properati['price_per_m2'],q=cant_buckets,labels=np.arange(cant_buckets))

[     0.      988.09   1976.18   2964.27   3952.36   4940.45   5928.54
   6916.63   7904.72   8892.81   9880.9   10868.99  11857.08  12845.17
  13833.26  14821.35  15809.44  16797.53  17785.62  18773.71  19761.8
  20749.89  21737.98  22726.07  23714.16  24702.25  25690.34  26678.43
  27666.52  28654.61  29642.7   30630.79  31618.88  32606.97  33595.06
  34583.15  35571.24  36559.33  37547.42  38535.51  39523.6   40511.69
  41499.78  42487.87  43475.96  44464.05  45452.14  46440.23  47428.32
  48416.41  49404.5   50392.59  51380.68  52368.77  53356.86  54344.95
  55333.04  56321.13  57309.22  58297.31  59285.4   60273.49  61261.58
  62249.67  63237.76  64225.85  65213.94  66202.03  67190.12  68178.21
  69166.3   70154.39  71142.48  72130.57  73118.66  74106.75  75094.84
  76082.93  77071.02  78059.11  79047.2   80035.29  81023.38  82011.47
  82999.56  83987.65  84975.74  85963.83  86951.92  87940.01  88928.1
  89916.19  90904.28  91892.37  92880.46  93868.55  94856.64  95844.73
  96832.

,created_on,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,...,title,dist_a_subte,dist_a_tren,dist_a_univ,dist_a_villa,dist_a_zona_anegada,categories_by_price,price_range,anio,is_new


In [290]:
#lo casteo a float porque si no tira error 
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 

properati.loc[:,('created_on')] = pd.to_datetime(properati['created_on'])
properati['anio'] = [x.year for x in properati['created_on'] ]

properati.loc[:,'is_new']=properati.loc[:,'description'].str.lower().str.contains('estrenar')

properati.groupby('categories_by_price').count().sort_values('created_on')

,created_on,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,...,description,title,dist_a_subte,dist_a_tren,dist_a_univ,dist_a_villa,dist_a_zona_anegada,price_range,anio,is_new
categories_by_price,,,,,,,,,,,,,,,,,,,,,
98.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
97.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
79.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
94.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
93.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
64.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
91.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
68.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
83.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [245]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'],properati['dist_a_tren'],properati['dist_a_villa'],\
        properati['dist_a_zona_anegada'],properati['surface_total_in_m2'],\
        properati['surface_covered_in_m2'],properati['lat'],properati['lon']])
y = properati['categories_by_price']

### Busco hiper-parametros con random search

In [171]:
%%notify

rf = RandomForestClassifier(n_jobs=-1)

oob_score=[True,False]
criterion = ['gini','entropy']
max_features =["auto","log2",None]
n_estimators = np.arange(10,100,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators,"oob_score":oob_score}

search = RandomizedSearchCV(rf, param_distributions=param_grid ,cv=20,n_iter=5) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 118.34 segundos para 5 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.992 (std: 0.001)
Promedio validation score: 0.375 (std: 0.045)
Promedio fit time: 1.578s
Hiper-parametros: {'n_estimators': 60, 'max_features': None, 'oob_score': True, 'criterion': 'entropy'}

Puesto: 2
Promedio training score: 0.992 (std: 0.001)
Promedio validation score: 0.370 (std: 0.040)
Promedio fit time: 0.818s
Hiper-parametros: {'n_estimators': 70, 'max_features': 'auto', 'oob_score': False, 'criterion': 'entropy'}

Puesto: 3
Promedio training score: 0.992 (std: 0.001)
Promedio validation score: 0.369 (std: 0.043)
Promedio fit time: 0.591s
Hiper-parametros: {'n_estimators': 50, 'max_features': 'log2', 'oob_score': False, 'criterion': 'entropy'}

Puesto: 4
Promedio training score: 0.992 (std: 0.001)
Promedio validation score: 0.361 (std: 0.038)
Promedio fit time: 0.961s
Hiper-parametros: {'n_estimators': 90, 'max_features': 'log2', 'oob_score': False, 'criterion': 'entro

<IPython.core.display.Javascript object>

In [172]:
mejor_rf = search.best_estimator_
print mejor_rf.feature_importances_ # la importancia de cada feature (cuanta informacion le provee al rf) ordenado tal y cual esta zipeado el X

[ 0.05679378  0.0469424   0.11615543  0.06550298  0.09035526  0.26879457
  0.15110547  0.12768706  0.07666306]


In [173]:
search.score(X_test,y_test) # pruebo el estimador refiteado con el set de test

0.35208711433756806

# RF Regresor

In [291]:
#preparo set de datos
X = zip(properati['dist_a_tren'],\
        properati['surface_total_in_m2'],\
        properati['surface_covered_in_m2'],properati['lat'],properati['lon'],properati['categories_by_price'])
y = properati['price_aprox_local_currency']

In [265]:
%%notify

rf = RandomForestRegressor(n_jobs=-1)

criterion = ['mse','mae']
max_features =["auto","log2",None]
n_estimators = np.arange(10,200,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

search = RandomizedSearchCV(rf, param_distributions=param_grid ,cv=20,n_iter=50) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

### mae tarda muchos mas que mse y log 2 anda mejor (mejor cv scor 0.76 )

### Hago GS en los mejores

In [292]:
%%notify

rf = RandomForestRegressor(n_jobs=-1)

criterion = ['mse']
max_features =["log2"]
n_estimators = np.arange(100,300,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

search = GridSearchCV(rf, param_grid=param_grid ,cv=20) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 366.66 segundos para 20 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.978 (std: 0.004)
Promedio validation score: 0.897 (std: 0.129)
Promedio fit time: 0.352s
Hiper-parametros: {'max_features': 'log2', 'n_estimators': 100, 'criterion': 'mse'}

Puesto: 2
Promedio training score: 0.980 (std: 0.003)
Promedio validation score: 0.896 (std: 0.131)
Promedio fit time: 0.644s
Hiper-parametros: {'max_features': 'log2', 'n_estimators': 200, 'criterion': 'mse'}

Puesto: 3
Promedio training score: 0.980 (std: 0.003)
Promedio validation score: 0.895 (std: 0.132)
Promedio fit time: 0.919s
Hiper-parametros: {'max_features': 'log2', 'n_estimators': 290, 'criterion': 'mse'}

Puesto: 4
Promedio training score: 0.980 (std: 0.003)
Promedio validation score: 0.894 (std: 0.134)
Promedio fit time: 0.576s
Hiper-parametros: {'max_features': 'log2', 'n_estimators': 160, 'criterion': 'mse'}

Puesto: 5
Promedio training score: 0.980 (std: 0.003)
Promedio validation score: 0

<IPython.core.display.Javascript object>

In [267]:
mejor_rf = search.best_estimator_
print mejor_rf.feature_importances_ # la importancia de cada feature (cuanta informacion le provee al rf) ordenado tal y cual esta zipeado el X

[ 0.09424034  0.21306946  0.31991499  0.09447475  0.07060633  0.20769412]


In [268]:
search.score(X_test,y_test)

0.69080782687795539

In [269]:
errores = mejor_rf.predict(X_test)-y_test
print ("Error maximo:{0}\nError minimo:{1}".format( max(abs(errores)),min(abs(errores))))
print errores

Error maximo:109726244.25
Error minimo:0.0
37473   -2.138840e+06
39875    1.725560e+06
15360    0.000000e+00
17841    2.006245e+04
13093    1.080170e+06
13541   -2.008336e+06
6952     2.808899e+05
11199    2.573862e+06
19579   -1.563760e+06
46855   -1.098795e+05
30214   -2.971236e+06
42251    1.626520e+06
28198    1.800066e+05
33328   -2.073556e+05
40972    6.329801e+05
9152    -1.970858e+06
14893    2.319925e+04
19041    9.880920e+04
24770    7.558708e+05
9619    -1.114871e+05
48122   -1.516970e+06
34965    0.000000e+00
30596   -3.404557e+05
40462    8.812230e+05
6772     5.195325e+04
18248   -3.465184e+06
2907    -8.517719e+05
14697   -4.119566e+05
18790    2.733133e+05
22010    2.331492e+06
             ...     
14176    3.310291e+06
9560     2.139820e+05
41407   -2.698955e+04
21738    2.973937e+06
34964    0.000000e+00
30410   -7.232938e+05
34947    2.308162e+05
15002   -1.130511e+07
42593    4.306565e+04
15466   -1.045600e+04
6030    -4.546334e+05
24855   -1.117864e+06
33347    1.